In [28]:
# !pip install schedule
import schedule
import time
import smtplib
import re
from six.moves import urllib


def sendemail(from_addr,  password,login=None,
              subject='Hello', message='Hello',
              to_addr_list=None,
              smtpserver='smtp.gmail.com:587'):
    header  = 'From: %s' % from_addr
    if to_addr_list==None:
        to_addr_list=[from_addr]
    else:
        if type(to_addr_list)!=list:
            raise Exception('to_addr_list should be list')
    header += 'To: %s' % ','.join(to_addr_list)
    header += 'Subject: %s' % subject
    message = header + message
 
    server = smtplib.SMTP(smtpserver)
    server.starttls()
    if login==None:
        login=from_addr.split('@')[0]
    server.login(login,password)
    problems = server.sendmail(from_addr, to_addr_list, message)
    server.quit()


class PriceTracker_BH():
    def fit(self,RawPrice,Email,password,
            Product=('WD121KRYZ','WD 12TB Gold 7200 rpm SATA III 3.5" Internal Datacenter HDD'),
            WebKeyWord_Price='<div class="prices" data-selenium="prices">.*?<span class="itc-you-pay-price bold">.*?([\d\.]+).*?</span>'):
        self.RawPrice=RawPrice
        self.Email=Email
        self.password=password
#         self.Intervertion=CheckIntervention
        self.ProductID=Product[0]
        self.ProductName=Product[1]
        self.PriceRe=WebKeyWord_Price
    def GetPrice(self):
#         try:
#             print(ProductID,ProductName,PriceRe)
#         except:
        ProductID,ProductName,PriceRe=self.ProductID,self.ProductName,self.PriceRe
#             print(ProductID,ProductName,PriceRe)
        self.Url='https://www.bhphotovideo.com/c/search?Ntt=%s&N=0&InitialSearch=yes&sts=ma&Top+Nav-Search=' % ProductID
        Web=urllib.request.urlopen(self.Url)
        self.Source=Web.read()
        temp=re.sub('\n|\t','',self.Source.split(ProductName)[-1])
        self.NewPrice=float(re.search(PriceRe,temp).group(1))
    
    def ComparePrice(self):
        if self.NewPrice<self.RawPrice:
            Subject="%s's price change from %f to %f" %(self.ProductName,float(self.RawPrice),self.NewPrice)
            sendemail(from_addr=self.Email,password=self.password,subject=Subject,message=Subject)
#             print Subject
        
    def run(self):
        self.GetPrice()
        self.ComparePrice()
    def main(self):
        schedule.every().day.at("08:00").do(self.run)

        while True:
            schedule.run_pending()
            time.sleep(60) # wait one minute


In [29]:
a=PriceTracker_BH()

In [30]:
a.fit(Email='111',password='111',RawPrice=10000)

In [31]:
a.run()

WD 12TB Gold 7200 rpm SATA III 3.5" Internal Datacenter HDD's price change from 10000.000000 to 469.990000
